<a href="https://colab.research.google.com/github/Omkar985087/Move-With-Joy-Project/blob/main/Face_recongnition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install opencv-python-headless matplotlib

In [3]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import os
import matplotlib.pyplot as plt
from google.colab import files
from google.colab import drive

In [4]:
drive.mount('/content/drive')

def download_sample_dataset():

    !wget https://example.com/face_dataset.zip
    !unzip face_dataset.zip
    print("Dataset downloaded and extracted.")

def load_face_dataset(dataset_path):

    images = []
    labels = []

    for person_name in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, person_name)

        if os.path.isdir(person_path):
            for image_file in os.listdir(person_path):
                image_path = os.path.join(person_path, image_file)

                try:

                    img = cv2.imread(image_path)
                    img = cv2.resize(img, (128, 128))
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                    images.append(img / 255.0)
                    labels.append(person_name)
                except Exception as e:
                    print(f"Error processing {image_path}: {e}")

    X = np.array(images)
    y = np.array(labels)


    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    return X, y_encoded, label_encoder

Mounted at /content/drive


In [5]:
def create_face_recognition_model(num_classes):
    """
    Create a Convolutional Neural Network for face recognition

    Args:
        num_classes (int): Number of unique faces to recognize

    Returns:
        tf.keras.Model: Compiled deep learning model
    """
    model = Sequential([

        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D((2, 2)),


        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),


        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])


    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [6]:
def train_face_recognition_model(X, y, test_size=0.2, epochs=50):
    """
    Train face recognition model with data augmentation

    Args:
        X (numpy.ndarray): Input images
        y (numpy.ndarray): Encoded labels
        test_size (float): Proportion of test data
        epochs (int): Number of training epochs

    Returns:
        tuple: Trained model and training history
    """

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42
    )


    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True
    )

    # Create model
    num_classes = len(np.unique(y))
    model = create_face_recognition_model(num_classes)

    # Train model
    history = model.fit(
        datagen.flow(X_train, y_train, batch_size=32),
        validation_data=(X_test, y_test),
        epochs=epochs
    )

    return model, history

In [7]:
def evaluate_and_visualize(model, X_test, y_test, label_encoder):
    """
    Evaluate model performance and visualize results

    Args:
        model (tf.keras.Model): Trained face recognition model
        X_test (numpy.ndarray): Test images
        y_test (numpy.ndarray): Test labels
        label_encoder (LabelEncoder): Label encoder
    """
    # Model evaluation
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

    # Predictions
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)

    # Visualization
    plt.figure(figsize=(15, 10))
    for i in range(min(9, len(X_test))):
        plt.subplot(3, 3, i+1)
        plt.imshow(X_test[i])
        true_label = label_encoder.inverse_transform([y_test[i]])[0]
        pred_label = label_encoder.inverse_transform([predicted_classes[i]])[0]
        plt.title(f"True: {true_label}\nPred: {pred_label}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Face Detection Function
def detect_faces_in_image(image_path, model, label_encoder):
    """
    Detect and recognize faces in an image

    Args:
        image_path (str): Path to input image
        model (tf.keras.Model): Trained face recognition model
        label_encoder (LabelEncoder): Label encoder
    """
    # Load pre-trained Haar Cascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Read image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Extract face region
        face_roi = img[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (128, 128))
        face_roi = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)

        # Normalize and predict
        input_face = face_roi / 255.0
        input_face = np.expand_dims(input_face, axis=0)

        prediction = model.predict(input_face)
        predicted_class = np.argmax(prediction)
        confidence = np.max(prediction)

        # Get label
        label = label_encoder.inverse_transform([predicted_class])[0]

        # Draw rectangle and label
        color = (0, 255, 0)  # Green color
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        text = f"{label} ({confidence*100:.2f}%)"
        cv2.putText(img, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Display result
    plt.figure(figsize=(10, 8))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title("Face Detection and Recognition")
    plt.show()

In [ ]:
def main():
    # Step 1: Prepare Dataset (Manual Upload or Download)
    print("Upload your face dataset or use sample dataset")
    # Option 1: Manual Upload
    uploaded = files.upload()

    # Option 2: Download sample dataset
    # download_sample_dataset()

    # Dataset path (adjust based on your upload method)
    dataset_path = "/content/drive/MyDrive/dataset"

    # Load dataset
    X, y, label_encoder = load_face_dataset(dataset_path)

    # Train model
    model, history = train_face_recognition_model(X, y)

    # Evaluate model
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    evaluate_and_visualize(model, X_test, y_test, label_encoder)

    # Optional: Upload test image for face detection
    print("Upload a test image for face detection")
    test_image = files.upload()
    test_image_path = list(test_image.keys())[0]

    # Detect faces
    detect_faces_in_image(test_image_path, model, label_encoder)

# Run the main function
if __name__ == "__main__":
    main()

# Bonus: Save and load model
def save_model(model, label_encoder):
    """
    Save trained model and label encoder
    """
    # Save model
    model.save('/content/drive/MyDrive/face_recognition_model.h5')

    # Save label encoder
    import joblib
    joblib.dump(label_encoder, '/content/drive/MyDrive/label_encoder.pkl')
    print("Model and label encoder saved successfully!")

def load_model():
    """
    Load saved model and label encoder
    """
    from tensorflow.keras.models import load_model
    import joblib

    # Load model
    model = load_model('/content/drive/MyDrive/face_recognition_model.h5')

    # Load label encoder
    label_encoder = joblib.load('/content/drive/MyDrive/label_encoder.pkl')

    return model, label_encoder

Upload your face dataset or use sample dataset


Saving Akshay Kumar_0.jpg to Akshay Kumar_0.jpg
Saving Akshay Kumar_1.jpg to Akshay Kumar_1.jpg
Saving Akshay Kumar_2.jpg to Akshay Kumar_2.jpg
Saving Akshay Kumar_3.jpg to Akshay Kumar_3.jpg
